In [1]:
from pyspark.sql.types import LongType, StringType

from typedspark._core.column import Column
from typedspark import Schema


class A(Schema):
    a: Column[LongType]
    b: Column[StringType]

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.Builder().config("spark.ui.showConsoleProgress", "false").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/30 12:32:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/30 12:32:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
A.a

RecursionError: maximum recursion depth exceeded in comparison

In [7]:
from typedspark._utils.create_dataset import create_empty_dataset


df_a = create_empty_dataset(spark, A)
df_b = create_empty_dataset(spark, A)

df_a.join(df_b, A.a.str)
df_a.unionByName(df_b)

DataFrame[a: bigint, b: string]

In [2]:
import pandas as pd

(
    spark.createDataFrame(
        pd.DataFrame(
            dict(
                name=["Jack", "John", "Jane"],
                age=[20, 30, 40],
            )
        )
    )
    .createOrReplaceTempView("person_table")
)

/Users/nanneaben/opt/anaconda3/envs/typedspark/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Users/nanneaben/opt/anaconda3/envs/typedspark/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [3]:
from typedspark import load_table

df, Person = load_table(spark, "person_table")

In [4]:
Person.name.dtype

pyspark.sql.types.StringType

In [5]:
from typedspark import Schema, Column, StructType, create_partially_filled_dataset
from pyspark.sql.types import IntegerType

class Values(Schema):
    a: Column[IntegerType]
    b: Column[IntegerType]

class Container(Schema):
    values: Column[StructType[Values]]

create_partially_filled_dataset(
    spark,
    Container,
    {
        Container.values: create_partially_filled_dataset(
            spark,
            Values,
            {Values.a: [1, 2, 3]},
        ).collect(),
    }
).createOrReplaceTempView("structtype_table")

container, ContainerSchema = load_table(spark, "structtype_table")

In [6]:
ContainerSchema.values.dtype

typedspark._core.datatypes.StructType[typedspark._schema.schema.DynamicallyLoadedSchema]

In [7]:
Values.a == Values.b

Column<'(a = b)'>